In [1]:
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Adult

In [2]:
dataset_orig = pd.read_csv('../data/adult.data.csv')
dataset_orig = dataset_orig.dropna()


## Drop NULL values
dataset_orig = dataset_orig.dropna()

## Drop categorical features
dataset_orig = dataset_orig.drop(['workclass','fnlwgt','education','marital-status','occupation','relationship','native-country'],axis=1)

## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == ' Male', 1, 0)
dataset_orig['race'] = np.where(dataset_orig['race'] != ' White', 0, 1)
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == ' <=50K', 0, 1)


## Discretize age
dataset_orig['age'] = np.where(dataset_orig['age'] >= 70, 70, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 60 ) & (dataset_orig['age'] < 70), 60, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 50 ) & (dataset_orig['age'] < 60), 50, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 40 ) & (dataset_orig['age'] < 50), 40, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 30 ) & (dataset_orig['age'] < 40), 30, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 20 ) & (dataset_orig['age'] < 30), 20, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 10 ) & (dataset_orig['age'] < 10), 10, dataset_orig['age'])
dataset_orig['age'] = np.where(dataset_orig['age'] < 10, 0, dataset_orig['age'])

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)



X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR

dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

(9769, 8)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Sex

In [3]:
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][3] == 0: ## index of Sex is 3, Race is 2
        row_[0][3] = 1
    else:
        row_[0][3] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

8715 1054


# Race

In [4]:
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][2] == 0: ## index of Sex is 3, Race is 2
        row_[0][2] = 1
    else:
        row_[0][2] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

9536 233


# Compas

In [5]:
dataset_orig = pd.read_csv('../data/compas-scores-two-years.csv')



## Drop categorical features
## Removed two duplicate coumns - 'decile_score','priors_count'
dataset_orig = dataset_orig.drop(['id','name','first','last','compas_screening_date',
                                  'dob','age','juv_fel_count','decile_score',
                                  'juv_misd_count','juv_other_count','days_b_screening_arrest',
                                  'c_jail_in','c_jail_out','c_case_number','c_offense_date','c_arrest_date',
                                  'c_days_from_compas','c_charge_desc','is_recid','r_case_number','r_charge_degree',
                                  'r_days_from_arrest','r_offense_date','r_charge_desc','r_jail_in','r_jail_out',
                                  'violent_recid','is_violent_recid','vr_case_number','vr_charge_degree','vr_offense_date',
                                  'vr_charge_desc','type_of_assessment','decile_score','score_text','screening_date',
                                  'v_type_of_assessment','v_decile_score','v_score_text','v_screening_date','in_custody',
                                  'out_custody','start','end','event'],axis=1)

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'Female', 1, 0)
dataset_orig['race'] = np.where(dataset_orig['race'] != 'Caucasian', 0, 1)
dataset_orig['priors_count'] = np.where((dataset_orig['priors_count'] >= 1 ) & (dataset_orig['priors_count'] <= 3), 3, dataset_orig['priors_count'])
dataset_orig['priors_count'] = np.where(dataset_orig['priors_count'] > 3, 4, dataset_orig['priors_count'])
dataset_orig['age_cat'] = np.where(dataset_orig['age_cat'] == 'Greater than 45',45,dataset_orig['age_cat'])
dataset_orig['age_cat'] = np.where(dataset_orig['age_cat'] == '25 - 45', 25, dataset_orig['age_cat'])
dataset_orig['age_cat'] = np.where(dataset_orig['age_cat'] == 'Less than 25', 0, dataset_orig['age_cat'])
dataset_orig['c_charge_degree'] = np.where(dataset_orig['c_charge_degree'] == 'F', 1, 0)




## Rename class column
dataset_orig.rename(index=str, columns={"two_year_recid": "Probability"}, inplace=True)

# Here did not rec means 0 is the favorable lable
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == 0, 1, 0)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)

X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

(1443, 6)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Sex

In [6]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: ## index of Sex is 0, Race is 2
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

1166 277


# Race

In [7]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][2] == 0: ## index of Sex is 0, Race is 2
        row_[0][2] = 1
    else:
        row_[0][2] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

1443 0


# German

In [8]:
dataset_orig = pd.read_csv('../data/GermanData.csv')

## Drop categorical features
dataset_orig = dataset_orig.drop(['1','2','4','5','8','10','11','12','14','15','16','17','18','19','20'],axis=1)

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A91', 1, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A92', 0, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A93', 1, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A94', 1, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A95', 0, dataset_orig['sex'])

# mean = dataset_orig.loc[:,"age"].mean()
# dataset_orig['age'] = np.where(dataset_orig['age'] >= mean, 1, 0)
dataset_orig['age'] = np.where(dataset_orig['age'] >= 25, 1, 0)
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A30', 1, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A31', 1, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A32', 1, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A33', 2, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A34', 3, dataset_orig['credit_history'])

dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A61', 1, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A62', 1, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A63', 2, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A64', 2, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A65', 3, dataset_orig['savings'])

dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A72', 1, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A73', 1, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A74', 2, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A75', 2, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A71', 3, dataset_orig['employment'])



## ADD Columns
dataset_orig['credit_history=Delay'] = 0
dataset_orig['credit_history=None/Paid'] = 0
dataset_orig['credit_history=Other'] = 0

dataset_orig['credit_history=Delay'] = np.where(dataset_orig['credit_history'] == 1, 1, dataset_orig['credit_history=Delay'])
dataset_orig['credit_history=None/Paid'] = np.where(dataset_orig['credit_history'] == 2, 1, dataset_orig['credit_history=None/Paid'])
dataset_orig['credit_history=Other'] = np.where(dataset_orig['credit_history'] == 3, 1, dataset_orig['credit_history=Other'])

dataset_orig['savings=500+'] = 0
dataset_orig['savings=<500'] = 0
dataset_orig['savings=Unknown/None'] = 0

dataset_orig['savings=500+'] = np.where(dataset_orig['savings'] == 1, 1, dataset_orig['savings=500+'])
dataset_orig['savings=<500'] = np.where(dataset_orig['savings'] == 2, 1, dataset_orig['savings=<500'])
dataset_orig['savings=Unknown/None'] = np.where(dataset_orig['savings'] == 3, 1, dataset_orig['savings=Unknown/None'])

dataset_orig['employment=1-4 years'] = 0
dataset_orig['employment=4+ years'] = 0
dataset_orig['employment=Unemployed'] = 0

dataset_orig['employment=1-4 years'] = np.where(dataset_orig['employment'] == 1, 1, dataset_orig['employment=1-4 years'])
dataset_orig['employment=4+ years'] = np.where(dataset_orig['employment'] == 2, 1, dataset_orig['employment=4+ years'])
dataset_orig['employment=Unemployed'] = np.where(dataset_orig['employment'] == 3, 1, dataset_orig['employment=Unemployed'])


dataset_orig = dataset_orig.drop(['credit_history','savings','employment'],axis=1)
## In dataset 1 means good, 2 means bad for probability. I change 2 to 0
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == 2, 0, 1)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)

dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

(200, 12)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Sex

In [9]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: ## index of Sex is 0
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

191 9


# Default 

In [10]:
## Load dataset
dataset_orig = pd.read_csv('../data/default_of_credit_card_clients_first_row_removed.csv')



## Drop categorical features


## Change column values

dataset_orig['sex'] = np.where(dataset_orig['sex'] == 2, 0,1)

protected_attribute = 'sex'

## Drop NULL values
dataset_orig = dataset_orig.dropna()

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(6000, 25)


# Sex

In [11]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: ## index of Sex is 0
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

5989 11


# Heart-Health

In [12]:
dataset_orig = pd.read_csv('../data/processed.cleveland.data.csv')

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## calculate mean of age column
mean = dataset_orig.loc[:,"age"].mean()
dataset_orig['age'] = np.where(dataset_orig['age'] >= mean, 0, 1)

## Make goal column binary
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] > 0, 1, 0)

protected_attribute = 'age'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

(60, 14)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Age

In [13]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: ## index of Age is 0
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

55 5


# Bank

In [14]:
dataset_orig = pd.read_csv('../data/bank.csv')

## Drop categorical features

dataset_orig = dataset_orig.drop(['job','marital','education','contact','month','poutcome'],axis=1)

dataset_orig['default'] = np.where(dataset_orig['default'] == 'no', 0, 1)
dataset_orig['housing'] = np.where(dataset_orig['housing'] == 'no', 0, 1)
dataset_orig['loan'] = np.where(dataset_orig['loan'] == 'no', 0, 1)
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == 'yes', 1, 0)

mean = dataset_orig.loc[:,"age"].mean()
dataset_orig['age'] = np.where(dataset_orig['age'] >= 30, 1, 0)


protected_attribute = 'age'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

(2233, 11)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Age

In [15]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: ## index of Age is 0
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

1787 446


# Home Credit

In [16]:
dataset_orig = pd.read_csv('../data/Home Credit Default Risk.csv')

## Drop categorical features

# dataset_orig = dataset_orig.fillna(0)


dataset_orig = dataset_orig.drop(['SK_ID_CURR','NAME_CONTRACT_TYPE','FLAG_OWN_CAR','FLAG_OWN_REALTY',
                                  'NAME_TYPE_SUITE','NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE' ,
                                  'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE','OCCUPATION_TYPE',
                                  'WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE',
                                  'HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE',
                                  'DAYS_LAST_PHONE_CHANGE'],axis=1)

# dataset_orig = dataset_orig.dropna()
dataset_orig = dataset_orig.fillna(0)
# ## Change symbolics to numerics
dataset_orig['CODE_GENDER'] = np.where(dataset_orig['CODE_GENDER'] == 'M', 1, 0)

protected_attribute = 'CODE_GENDER'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(61503, 105)


# Sex

In [17]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][1] == 0: ## index of Age is 0
        row_[0][1] = 1
    else:
        row_[0][1] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

37290 24213


# Student

In [18]:
## Load dataset
dataset_orig = pd.read_csv('../data/student/Student.csv')

## Drop NULL values
dataset_orig = dataset_orig.dropna()

## Drop categorical features
dataset_orig = dataset_orig.drop(['school','address', 'famsize', 'Pstatus','Mjob', 'Fjob', 'reason', 'guardian'],axis=1)

## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'M', 1, 0)
dataset_orig['schoolsup'] = np.where(dataset_orig['schoolsup'] == 'yes', 1, 0)
dataset_orig['famsup'] = np.where(dataset_orig['famsup'] == 'yes', 1, 0)
dataset_orig['paid'] = np.where(dataset_orig['paid'] == 'yes', 1, 0)
dataset_orig['activities'] = np.where(dataset_orig['activities'] == 'yes', 1, 0)
dataset_orig['nursery'] = np.where(dataset_orig['nursery'] == 'yes', 1, 0)
dataset_orig['higher'] = np.where(dataset_orig['higher'] == 'yes', 1, 0)
dataset_orig['internet'] = np.where(dataset_orig['internet'] == 'yes', 1, 0)
dataset_orig['romantic'] = np.where(dataset_orig['romantic'] == 'yes', 1, 0)
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] > 12, 1, 0)




protected_attribute = 'sex'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

(209, 25)


c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Sex

In [19]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][0] == 0: ## index of Sex is 0
        row_[0][0] = 1
    else:
        row_[0][0] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

201 8


# MEPS 15

In [20]:
dataset_orig = pd.read_csv('../data/MEPS/h181.csv')

# ## Drop NULL values
dataset_orig = dataset_orig.dropna()


dataset_orig = dataset_orig.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT15' : 'POVCAT', 'INSCOV15' : 'INSCOV'})


dataset_orig = dataset_orig[dataset_orig['PANEL'] == 20]
dataset_orig = dataset_orig[dataset_orig['REGION'] >= 0] # remove values -1
dataset_orig = dataset_orig[dataset_orig['AGE'] >= 0] # remove values -1
dataset_orig = dataset_orig[dataset_orig['MARRY'] >= 0] # remove values -1, -7, -8, -9
dataset_orig = dataset_orig[dataset_orig['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
dataset_orig = dataset_orig[(dataset_orig[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                             'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                             'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                             'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                             'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]

# ## Change symbolics to numerics
dataset_orig['RACEV2X'] = np.where((dataset_orig['HISPANX'] == 2 ) & (dataset_orig['RACEV2X'] == 1), 1, dataset_orig['RACEV2X'])
dataset_orig['RACEV2X'] = np.where(dataset_orig['RACEV2X'] != 1 , 0, dataset_orig['RACEV2X'])
dataset_orig = dataset_orig.rename(columns={"RACEV2X" : "RACE"})
# dataset_orig['UTILIZATION'] = np.where(dataset_orig['UTILIZATION'] >= 10, 1, 0)



def utilization(row):
        return row['OBTOTV15'] + row['OPTOTV15'] + row['ERTOT15'] + row['IPNGTD15'] + row['HHTOTD15']

dataset_orig['TOTEXP15'] = dataset_orig.apply(lambda row: utilization(row), axis=1)
lessE = dataset_orig['TOTEXP15'] < 10.0
dataset_orig.loc[lessE,'TOTEXP15'] = 0.0
moreE = dataset_orig['TOTEXP15'] >= 10.0
dataset_orig.loc[moreE,'TOTEXP15'] = 1.0

dataset_orig = dataset_orig.rename(columns = {'TOTEXP15' : 'UTILIZATION'})

dataset_orig = dataset_orig[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT15F']]

dataset_orig = dataset_orig.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})
protected_attribute = 'race'


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(3514, 43)


# Race

In [21]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][3] == 0: ## index of race is 3
        row_[0][3] = 1
    else:
        row_[0][3] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

3428 86


# MEPS 16 

In [22]:
dataset_orig = pd.read_csv('../data/MEPS/h192.csv')

# ## Drop NULL values
dataset_orig = dataset_orig.dropna()


dataset_orig = dataset_orig.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT16' : 'POVCAT', 'INSCOV16' : 'INSCOV'})


dataset_orig = dataset_orig[dataset_orig['PANEL'] == 21]
dataset_orig = dataset_orig[dataset_orig['REGION'] >= 0] # remove values -1
dataset_orig = dataset_orig[dataset_orig['AGE'] >= 0] # remove values -1
dataset_orig = dataset_orig[dataset_orig['MARRY'] >= 0] # remove values -1, -7, -8, -9
dataset_orig = dataset_orig[dataset_orig['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
dataset_orig = dataset_orig[(dataset_orig[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                         'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                         'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                         'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                         'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]

# ## Change symbolics to numerics
dataset_orig['RACEV2X'] = np.where((dataset_orig['HISPANX'] == 2 ) & (dataset_orig['RACEV2X'] == 1), 1, dataset_orig['RACEV2X'])
dataset_orig['RACEV2X'] = np.where(dataset_orig['RACEV2X'] != 1 , 0, dataset_orig['RACEV2X'])
dataset_orig = dataset_orig.rename(columns={"RACEV2X" : "RACE"})
# dataset_orig['UTILIZATION'] = np.where(dataset_orig['UTILIZATION'] >= 10, 1, 0)



def utilization(row):
        return row['OBTOTV16'] + row['OPTOTV16'] + row['ERTOT16'] + row['IPNGTD16'] + row['HHTOTD16']

dataset_orig['TOTEXP16'] = dataset_orig.apply(lambda row: utilization(row), axis=1)
lessE = dataset_orig['TOTEXP16'] < 10.0
dataset_orig.loc[lessE,'TOTEXP16'] = 0.0
moreE = dataset_orig['TOTEXP16'] >= 10.0
dataset_orig.loc[moreE,'TOTEXP16'] = 1.0

dataset_orig = dataset_orig.rename(columns = {'TOTEXP16' : 'UTILIZATION'})

dataset_orig = dataset_orig[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT16F']]

dataset_orig = dataset_orig.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})
protected_attribute = 'race'


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)


dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X, y= dataset_orig.loc[:, dataset_orig.columns != 'Probability'], dataset_orig['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf.fit(X_train, y_train)
print(dataset_orig_test.shape)

c:\users\jchakra\python\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(3135, 43)


# Race

In [23]:
# Create new test by switching the value of prottected attribute
same , not_same = 0,0
for index,row in dataset_orig_test.iterrows():
    row_ = [row.values[0:len(row.values)-1]]    
    y_normal = clf.predict(row_)
    # Here protected attribute value gets switched
    if row_[0][3] == 0: ## index of race is 3
        row_[0][3] = 1
    else:
        row_[0][3] = 0    
    y_reverse = clf.predict(row_)
    if y_normal[0] != y_reverse[0]:
        not_same += 1
    else:
        same += 1
print(same , not_same)

3049 86
